# **Winelists**

In [ ]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://nairobidrinks.co.ke/pricelist'
r = requests.get(url)

<Response [200]>

In [ ]:
soup = BeautifulSoup(r.text, 'lxml')
sects = soup.find_all('tr')

In [ ]:
titles = []
for tr in sects:
    title = tr.find_all('td')
    titles.extend(title)

In [ ]:
products = [title.text.strip() for title in titles]
caps = products[0::3]
prices = products[1::3]

In [ ]:
prod_split = [cap.split('\n') for cap in caps]
names = [split[0] for split in prod_split]
vol = [split[1] for split in prod_split]

full_list = {
    'Name': names,
    'Volume': vol,
    'Price': prices
}

In [ ]:
df = pd.DataFrame(full_list)
df.drop_duplicates()
df.to_csv('Liqourlist.csv', index=False)

In [ ]:
viewdf = pd.read_csv('Liqourlist.csv')
viewdf

,Name,Volume,Price
0,Jack Daniels Tennessee Cola,330 ml,Ksh 460
1,Best whisky,750 ml,"Ksh 1,400"
2,Black & white whisky,750 ml,"Ksh 1,499"
3,William lawson,750 ml,"Ksh 1,800"
4,Sir Edwards,750 ml,"Ksh 1,800"
...,...,...,...
2581,Dom perignon brut luminous,750 ml,"Ksh 59,999"
2582,Dom perignon brut 20.. vintage,750 ml,"Ksh 59,999"
2583,Dom perignon rose,750 ml,"Ksh 64,999"
2584,Armand de brignac ace of spades brut gold cham...,750 ml,"Ksh 64,999"


# **Wholesalers**

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://jayswines.com/categories/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

urlists = soup.find_all('a', {'class':'vc_column-link smooth-scroll'})
links = [url.get('href') for url in urlists]
links = links[:-2]

for link in links:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')

    # Extract product names and prices
    wraps = soup.find_all('div', {'class':'w-vwrapper usg_vwrapper_1 align_center valign_top'})
    names = [wrap.find('a') for wrap in wraps]
    name = [name.text.strip() for name in names]

    price_spans = [wrap.find('span', {'class':'woocommerce-Price-amount amount'}) for wrap in wraps]
    price = [price_span.text.strip() for price_span in price_spans]

    data = {
        'Name': name,
        'Price': price
    }

    df = pd.DataFrame(data)
    df.drop_duplicates()

    # Get the name from the link
    product_name = link.split('/')[-2]
    product_name = product_name.upper()

    # Save the data to a CSV file
    df.to_csv(f'{product_name}.csv', index=False)

In [5]:
pd.read_csv('WINE.csv')

,Name,Price
0,SIMONSIG FRANS MALAN (COMPLEX CAPE STYLE BLEND...,"KSh6,750.00"
1,NAPA CELLARS CABERNET SAUVIGNON 750ML (14.2%),"KSh6,400.00"
2,NAPA CELLARS CHARDONNY 750ML (14%),"KSh6,400.00"
3,SIMONSIG TIARA DRY RED 750ML(14.5%),"KSh6,400.00"
4,GRAHAMS FINE 10YEARS TAWNY PORT 750ML(20%),"KSh6,200.00"
...,...,...
594,KEROR SPARKLING RED 750ML(0%),KSh540.00
595,RENDEZ VOUS SPARKLING WHITE NON-ALCOHOLIC 750M...,KSh520.00
596,RENDEZ VOUS SPARKLING RED NON-ALCOHOLIC 750ML(0%),KSh400.00
597,RENDEZ VOUS SPARKLING PINK NON-ALCOHOLIC 750ML...,KSh520.00
